### Imports

In [1]:
import os
import pickle

from skimage import io
import pandas as pd
import cv2

from doctr.io import DocumentFile
from doctr.models import ocr_predictor

### Model Loading

In [2]:
model = ocr_predictor(pretrained=True)

/home/venkat/BADRI/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/venkat/BADRI/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


### DocTR Predictions

In [19]:
input_dir   = './../../../docbank_processed/processed_data/only_eqn/ori_black/'
output_dir  = './../../../docbank_processed/processed_data/only_eqn/doctr/'

In [20]:
dir_list = os.listdir(input_dir)
results = {}
images = []

for image in dir_list:
    doc = DocumentFile.from_images(input_dir+image)
    result = model(doc)
    results[image] = result
    images.append(image)

In [24]:
dimensions = {}
total = {}
for image,result in results.items():
    dim = tuple(reversed(result.pages[0].dimensions))
    predictions = []
    img = io.imread(os.path.join(input_dir,image))
    for block in result.pages[0].blocks:
        for line in block.lines:
            for word in line.words:
                values = []
                geo = word.geometry
                a = list(int(a*b) for a,b in zip(geo[0],dim))
                b = list(int(a*b) for a,b in zip(geo[1],dim))
                values.append(a[0])
                values.append(a[1])
                values.append(b[0]-a[0])
                values.append(b[1]-a[1])
                values.append(word.value)
                values.append(word.confidence)
                predictions.append(values)
                cv2.rectangle(img, (a[0], a[1]), (b[0], b[1]), (0, 0, 255))
    io.imsave(output_dir + 'images/' + image,img)
    total[image] = predictions

/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/113.tar_1507.06116.gz_fluct_150720_7_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)
/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/232.tar_1808.04097.gz_ep_LHC_submit_22_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)
/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/44.tar_1503.06300.gz_dodona_ijhcs_revised_round2_6_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)
/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/7.tar_1801.02983.gz_Article_7_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)
/tmp/ipykernel_848373/320478551

/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/63.tar_1504.07006.gz_mayak_arxiv_20141204_7_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)
/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/187.tar_1511.05780.gz_Levy_irregular_sampling_5_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)
/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/152.tar_1509.08018.gz_chaindecodingTCOM_v10_69_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)
/tmp/ipykernel_848373/3204785517.py:22: UserWarning: ./../../../docbank_processed/processed_data/only_eqn/doctr/images/89.tar_1407.4134.gz_NMSSM_EWPT_submission_2_26_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)


### Store Results

In [25]:
for image,result in total.items():
    name = image[:len(image) - 12]
    df = pd.DataFrame(result, columns = ['x0', 'y0', 'w', 'h', 'prediction', 'confidence'])
    df.to_csv(output_dir + 'txt/' + name + '.txt', sep='\t',index=False)